**Design principles**

- Cascades of multisite phosphorylation generate ultrasensitivity

**Concepts**

- Sensitivity amplification

<hr>

In [10]:
# Colab setup ------------------
import os, sys, subprocess
if "google.colab" in sys.modules:
    cmd = "pip install --upgrade watermark"
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
# ------------------------------

import numpy as np
import biocircuits
import bokeh.io
import bokeh.plotting

bokeh.io.output_notebook()

Loading BokehJS ...

The near-perfect linear responses described in the previous chapter can be extremely useful. However, the opposite behavior is also important. Cells sometimes need to respond in an all-or-none manner to an external stimulus. Should the cell divide? Should it differentiate? Should it undergo cell death? All of these questions require yes-or-no answers in response to "fuzzy" inputs. 

When we introduced ultrasensitivty in Chapters 2 and 3, via the Hill function, we pointed out that it could be achieved through cooperativity at the molecular level, for instance through assembly of multimeric complexes. However, this mechanism has limitations. Generating very ultrasensitive responses ($n > 10$, for example) using only multimerization would require large complexes with many subunits operating in regimes of low concentration or weak binding. This constrains protein designs and requires the system to operate in a regime of excess total protein. Additionally, it would be useful if ultrasensitivity were a tunable property. Then cells could respond ultrasensitively to a signal, such as a hormone, in one context, but more linearly in a different context. 

In this chapter, we will explore how a ubiquitous eukaryotic pathway—the iconic MAP kinase cascade—can achieve diverse, tunable input-output responses, including high levels of ultrasensitivity. 

## MAP Kinase cascades and the circuitry of ultrasensitivity

In contrast to the prokaryotic two-component systems of the previous chapter, eukaryotic cells prominently feature a different type of kinase cascade, known as the mitogen-activated protein kinase (MAP Kinase) cascade. MAP kinase cascades are found in yeast, plants, and mammals, duplicated in some species, embedded within larger circuits, and elaborated with additional regulatory inputs and feedbacks. 

In the canonical MAP kinase pathway, signals are transmitted from an upstream "kinase kinase kinase" (MAPKKK) to a "kinase kinase" (MAPKK) to a "kinase" (MAPK), which phosphorylates target proteins. At each step, the upstream kinase phosphorylates _two_ distinct sites on its substrate kinase, activating the substrate's enzymatic kinase activity. 

<div style="width: 450px; margin: auto;">
    
![MAP kinase cascade schematic](figs/MAPK_cascades.png)

</div>

This pervasive kinase cascade has been shown to amplify signals, accelerate response times, integrate multiple input signals, and enhance noise tolerance. Perhaps most dramatically, it can generate ultrasensitive, switch-like responses (see [Huang and Ferrel 1996, PNAS](https://doi.org/10.1073/pnas.93.19.10078)). James Ferrell and colleagues did pioneering work to [quantitatively understand](http://dx.doi.org/10.1016/j.tibs.2014.10.002) the nature of amplification in kinase cascades. This and other work has shown how the MAP kinase cascade can provide tunable sensitivity amplification and many other interesting dynamical behaviors. ([Bhalla et al 2002](https://doi.org/10.1126/science.1068873); [Poritz et al, 2007](https://doi.org/10.1002/yea.777)).

These properties emerge from two different aspects of the MAP kinase cascade, both evident in the diagram above: multisite phosphorylation and the cascading multi-step architecture. What does each feature contribute? Why do we need both of them? Gazing at the diagram above, the answers to these questions are not immediately apparent. It is also not obvious what kinds of input-output responses are possible, how they depend on the abundances of the different protein components and biochemical parameters of the various enzymatic reactions. To gain insight, we will proceed systematically, starting with a single phosphorylation site, progressing to multi-site phosphorylation, and then considering cascades of multisite kinase reactions. At each stage, we will focus primarily on steady-state responses. However, analysis of kinase cascades in individual cells has revealed that they can be stunningly dynamic in some contexts (For instance, check out Movie 1 in [Raina et al, 2022](https://doi.org/10.1242/dev.199710)). 

## Single-site phosphorylation is not ultrasensitive

Consider a minimal phosphorylation system, in which an input signal leads to phosphorylation of a substrate on a single site. The substrate can also be dephosphorylated. We will denote the unphosphorylated (phosphorylated) form as $\mathrm{X}_{10}$ ($\mathrm{X}_{11}$), where the first subscript will later be used to allow multiple kinases when we consider cascades, and the second subscript represents the number of phosphorylations on the protein. 

\begin{align}
\require{mhchem}
\ce{X_{10} <=>[\mathrm{S}] X_{11}}
\end{align}

The rate of phosphorylation can be described by [Michaelis-Menten kinetics](06a_michaelis_menten.ipynb).

\begin{align}
\text{rate of phosphorylation} = k_\mathrm{cat} s \,\frac{x_{10}/K_M}{1 + x_{10}/K_M},
\end{align}

where $K_M$ is the Michaelis constant, and $x_{10}$ denotes the concentration of $\mathrm{X}_{10}$. In this chapter, we will focus on regimes far from saturation, where the concentration of substrates is small relative to $K_M$. In this limit,

\begin{align}
\text{rate of phosphorylation} \approx k_+ s x_{10},
\end{align}

where $k_+ = k_\mathrm{cat}/K_M$.

Including dephosphorylation leads to the following ODE. 

\begin{align}
\frac{\mathrm{d}x_{11}}{\mathrm{d}t} = k_+ s x_{10} - k_- x_{11}.
\end{align}

where $k_-$ is the rate constant for dephosphorylation, which is assumed to be catalyzed by various cellular phosphatases. 

Defining the total concentration of $\mathrm{X}_1$ as $x_1^\mathrm{tot} = x_{10}+x_{11}$, we obtain the steady-state concentration of phosphorylated protein

\begin{align}
x_{11} = x_1^\mathrm{tot}\,\frac{k_+ s}{k_+ s + k_-} = x_1^\mathrm{tot}\,\frac{s/K}{1 + s/K},
\end{align}

where $K \equiv k_- / k_+$. 

Henceforth, we will non-dimensionalize using $K$ as the unit of concentration, so that

\begin{align}
x_{11} = x_1^\mathrm{tot}\,\frac{s}{1 + s},
\end{align}


This equation should seem very familiar, since similar expressions have popped up over and over again starting with the DNA binding relationship in [Chapter 1](01_intro_to_circuit_design.ipynb), and also very boring. So far, nothing interesting is happening! 

### Double phosphorylation allows ultrasensitive responses

In the MAP kinase cascade diagrammed above, you can see that some components can be phosphorylated multiple times, on distinct sites. Further, these components are fully catalytically active only when _doubly_ phosphorylated. The requirement for double phosphorylation (or, more generally, multiple phosphorylation) provides a mechanism for ultrasensitivity. To see why, let's extend our system to two phosphorylations:

\begin{align}
\require{mhchem}
\ce{X_{10} <=>[\mathrm{S}] X_{11} <=>[\mathrm{S}] X_{12} }
\end{align}

with $\mathrm{X}_{12}$ being active, and its other forms inactive.

As derived in the [technical appendix](../technical_appendices/07a_double_phos_double_cascade.ipynb), the steady-state concentration of the doubly phosphorylated form of $\mathrm{X}$ is given by

\begin{align}
x_{12} = x_1^\mathrm{tot}\,\frac{s^2}{1 + s + s^2}.
\end{align}

This is *not* a Hill function—note the extra $s$ in the denominator—but it *is* ultrasensitive. Now we are getting somewhere.


In [11]:
# Curves for plots
s = np.logspace(-4, 4, 400)
tf_baseline = s / (1 + s)
tf_twophos = s ** 2 / (1 + s + s ** 2)
gain_baseline = tf_baseline / s
gain_twophos = tf_twophos / s
deriv_baseline = 1 / (1 + s) ** 2
deriv_twophos = s * (2 + s) / (1 + s + s ** 2) ** 2
sens_baseline = s / tf_baseline * deriv_baseline
sens_twophos = s / tf_twophos * deriv_twophos

# Build eight figure (tf, gain, deriv, sens), (log, linear)
fig_kwargs = dict(
   frame_width=300,
   frame_height=175,
   x_axis_type="log",
   x_axis_label="s",
   x_range=[s.min(), s.max()],
)
titles = dict(
   tf="transfer function", gain="gain", deriv="derivative", sens="sensitivity"
)
y_axis_labels = dict(
    tf=r"$$\text{trans. fun.} / x_1^\mathrm{tot}$$", 
    gain=r"$$\text{gain} / x_1^\mathrm{tot}$$", 
    deriv=r"$$\text{derivative} / x_1^\mathrm{tot}$$", 
    sens="sensitivity"
)
plots = {
   feature: bokeh.plotting.figure(
       **fig_kwargs,
       y_axis_label=y_axis_labels[feature],
       title=titles[feature],
       visible=False,
   )
   for feature in ["tf", "gain", "deriv", "sens"]
}

log_plots = {
   feature: bokeh.plotting.figure(
       **fig_kwargs,
       y_axis_type="log",
       y_axis_label=y_axis_labels[feature],
       title=titles[feature],
   )
   for feature in ["tf", "gain", "deriv", "sens"]
}
for feature in log_plots:
   log_plots[feature].y_range.start = 1e-4

# Link x-ranges
for key, p in plots.items():
   if key != "tf":
       p.x_range = plots["tf"].x_range
   log_plots[key].x_range = plots["tf"].x_range

# Plot baselines
plots["tf"].line(s, tf_baseline, color="gray", line_width=2, legend_label='single phos.')
log_plots["tf"].line(s, tf_baseline, color="gray", line_width=2, legend_label='single phos.')
plots["deriv"].line(s, deriv_baseline, color="gray", line_width=2)
log_plots["deriv"].line(s, deriv_baseline, color="gray", line_width=2)
plots["gain"].line(s, gain_baseline, color="gray", line_width=2)
log_plots["gain"].line(s, gain_baseline, color="gray", line_width=2)
plots["sens"].line(s, sens_baseline, color="gray", line_width=2)
log_plots["sens"].line(s, sens_baseline, color="gray", line_width=2)

# Plot two phosphorylation sides
plots["tf"].line(s, tf_twophos, line_width=2, legend_label='double phos.')
log_plots["tf"].line(s, tf_twophos, line_width=2, legend_label='double phos.')
plots["deriv"].line(s, deriv_twophos, line_width=2)
log_plots["deriv"].line(s, deriv_twophos, line_width=2)
plots["gain"].line(s, gain_twophos, line_width=2)
log_plots["gain"].line(s, gain_twophos, line_width=2)
plots["sens"].line(s, sens_twophos, line_width=2)
log_plots["sens"].line(s, sens_twophos, line_width=2)

# Put legend out of the way
plots["tf"].legend.location = 'bottom_right'
log_plots["tf"].legend.location = 'bottom_right'

# Widget for switching log vs linear
radio_button_group = bokeh.models.RadioButtonGroup(
   labels=["log", "linear"],
   active=0,
   width=100,
)
radio_button_group.js_on_change(
   "active",
   bokeh.models.CustomJS(
       args=dict(
           **{
               "p_" + feature: plots[feature]
               for feature in ["tf", "gain", "deriv", "sens"]
           },
           **{
               "p_" + feature + "_log": log_plots[feature]
               for feature in ["tf", "gain", "deriv", "sens"]
           },
       ),
       code="""
 if (p_tf_log.visible == true) {
   p_tf_log.visible = false;
   p_tf.visible = true;
   p_gain_log.visible = false;
   p_gain.visible = true;
   p_deriv_log.visible = false;
   p_deriv.visible = true;
   p_sens_log.visible = false;
   p_sens.visible = true;
 }
 else {
   p_tf_log.visible = true;
   p_tf.visible = false;
   p_gain_log.visible = true;
   p_gain.visible = false;
   p_deriv_log.visible = true;
   p_deriv.visible = false;
   p_sens_log.visible = true;
   p_sens.visible = false;
 }
""",
   ),
)

# Build layout
layout = bokeh.layouts.row(
   bokeh.layouts.column(
       bokeh.layouts.row(
           bokeh.layouts.Spacer(width=20),
           radio_button_group,
       ),
       bokeh.layouts.Spacer(height=15),
       bokeh.layouts.row(
           bokeh.layouts.column(plots["tf"], log_plots["tf"]),
           bokeh.layouts.column(plots["gain"], log_plots["gain"]),
       ),
       bokeh.layouts.row(
           bokeh.layouts.column(plots["deriv"], log_plots["deriv"]),
           bokeh.layouts.column(plots["sens"], log_plots["sens"]),
       ),
   ),
   bokeh.layouts.Spacer(width=50),
)

bokeh.io.show(layout)

### Cascading kinases

Can we increase ultrasensitivity even further? The most obvious strategy would be to add more phosphorylation sites. Suppose the protein has four phosphorylation sites, with only the fully phosphorylated form active.

<div style="width: 450px; margin: auto;">
    
![4_site_phosphorylation](figs/4_site_phosphorylation.png)

</div>

Or, in reaction notation, 

\begin{align}
\require{mhchem}
\ce{X_{10} <=>[\mathrm{S}] X_{11} <=>[\mathrm{S}] X_{12} <=>[\mathrm{S}] X_{13} <=>[\mathrm{S}] X_{14} }.
\end{align}

A similar analysis, provided in the [technical appendix](../technical_appendices/07b_multiphos_multicascade.ipynb), shows that the concentration of the quadruply phosphorylated form is given by

\begin{align}
x_{14} = x_1^\mathrm{tot}\,\frac{s^4}{1 + s + s^2 + s^3 + s^4}.
\end{align}

This is indeed even more ultrasensitive. More is more.

But there is another way to sharpen this switch. Instead of expanding the degree of multisite phosphorylation in a single protein, as  you could create a multistep cascade in which one kinase phosphorylates the next. For example, here is a "2+2" design comprising two kinases, each phosphorylated at two sites.

<div style="width: 450px; margin: auto;">
    
![4_site_phosphorylation](figs/2_steps_2_phosphosites.png)

</div>



To understand how multi-step cascades impact sensitivity, first consider the simplest case of two kinases with single phosphorylation sites.

\begin{align}
\require{mhchem}
&\ce{X_{10}<=>[\mathrm{S}] X_{11}},\\[1em]
&\ce{X_{20}<=>[\mathrm{X}_{11}] X_{21}}.
\end{align}

The  [technical appendix](../technical_appendices/07a_double_phos_double_cascade.ipynb) shows that the output of this circuit, the concentration of $X_{21}$, is 

\begin{align}
x_{21} = x_2^\mathrm{tot}\,\frac{x_2^\mathrm{tot}\,s}{1 + s + x_1^\mathrm{tot}\,s}.
\end{align}

As you can see from the absence of exponents, cascading by itself does not provide ultrasensitivity where there was none to begin with. Will we do any better by cascading doubly phosphorylated proteins?


To find out, let's consider the "2+2" design diagrammed above, in which both $\mathrm{X}_1$ and $\mathrm{X}_2$  undergo double phosphorylation, and doubly phosphorylated $\mathrm{X}_1$ catalyzes phosphorylation of $\mathrm{X}_2$. 

\begin{align}
\require{mhchem}
&\ce{X_{10} <=>[\mathrm{S}] X_{11} <=>[\mathrm{S}] X_{12}},\\[1em]
&\ce{X_{20} <=>[\mathrm{X}_{12}] X_{21} <=>[\mathrm{X}_{12}] X_{22}}.
\end{align}

A straightforward analysis of the steady states of the corresponding ODEs reveals that 

\begin{align}
x_{22} = x_2^\mathrm{tot}\,\frac{(x_1^\mathrm{tot})^2 s^4}{(1+s+s^2)^2 + x_1^\mathrm{tot} s^2 (1 + s + s^2) + (x_1^\mathrm{tot})^2 s^4}.
\end{align}



You can see that this expression for $x_{22}$ has similarities and differences with that for $x_{14}$ above. On the one hand, both expressions depend on the 4th power of the input, $s$, in the numerator, and contain 4th order polynomials of $s$ in the denominator. On the other hand, they differ their dependence on the protein concentrations. $x_{14}$ is proportional to the total concentration of the single protein in the system, $x_1^\mathrm{tot}$, while $x_{22}$ is proportional to $x_2^\mathrm{tot}$, but has a more complex dependence on $x_1^\mathrm{tot}$. 

To better understand how these differences impact the transfer functions, gain, derivative and sensitivity of the two systems, it is helpful to play with the two systems interactively.

In [12]:
bokeh.io.show(biocircuits.jsplots.phosphorylation_signal_cascade_22_14())

AttributeError: module 'biocircuits.jsplots' has no attribute 'phosphorylation_signal_cascade_22_14'

Set all protein concentrations to 10. In this regime, we see that the two-step system (orange) saturates at a lower response (transfer function) but exhibits similar sensitivity and higher peak gain and derivative. In this regime, a striking advantage of the two-step system is that it activates at a lower level of the input. It's "EC50" value, the concentration of input, $s$, that produces a half-maximal response, is lower than that of the single layer 4-site system. The larger $x_1^\mathrm{tot}$, the stronger this EC50 advantage. This advantage is offset by a reduction, for some expression levels, in its maximum response, which becomes limited by $x_2^\mathrm{tot}$. By playing with the system, or just by looking at the expression above, you can see that $x_2^\mathrm{tot}$ linearly scales the output.

The absolute and relative abundances of MAP Kinase cascade proteins vary among cell types. The simple model above shows that this variation could allow the cascade to tune the input concentration to which it responds. By contrast the "1x4" system always responds at the same threshold, regardless of its protein concentration. 

## Saturation is a thing

The analysis above gave us good intuition about the relationships and tradeoffs between multisite phosphorylation and multi-step cascades. However, we have neglected a fundamental feature of real enzymatic systems: saturation. 

A single enzyme molecule can only phosphorylate so many substrates in a given time. As a result, the velocity of any enzymatic reaction eventually saturates at large enough substrate concentrations, as we saw in our discussion of [Michaelis-Menten kinetics](../technical_appendices/06a_michaelis_menten.ipynb). Further, the rate of phosphorylation is competing with a rate of dephosphorylation. These factors limit the maximum phosphorylation a given enzyme can produce. 

Multi-stage cascades, with increasing total protein concentrations at each level, can circumvent this limit. A lower abundance "upstream" kinase, $x_1^\mathrm{tot}$, can phosphorylate a higher abundance $x_2^\mathrm{tot}$, and so on. At each, the number of activated kinases increases, expanding the number of target molecules that can be phosphorylated. 

To see how this works, consider the modified interactive plot below, which incorporates saturation. Note that with saturation, the effects seen above are even more significant. For example, the difference in EC50s between the 1x4 and the 2x2 designs is more pronounced. 

**ADD INTERACTIVE PLOT WITH SATURATION HERE**

### Conclusions 

Multi-step phosphorylation cascades are ubiquitous in eukaryotic cells, where they play numerous central roles in diverse biological processes. In these systems, the combination of multisite phosphorylation and multi-step cascading together generate ultrasensitive responses. These responses can be tuned by adjusting the relative abundances of different proteins within the cascade. In particular, cascades allow systems to respond at lower input levels and to reach higher total activities for their endpoint kinases. They can thus be thought of as tunable amplifier modules. The systems considered above are quite simplified compared to their natural counterparts, which can have complex and variable architectures beyond those considered here.